In [ ]:
# Parse j301_5.sm file
sm_content = """************************************************************************
file with basedata            : j30_17.bas
initial value random generator: 1989763511
************************************************************************
projects                      :  1
jobs (incl. supersource/sink ):  32
horizon                       :  119
RESOURCES
  - renewable                 :  4   R
  - nonrenewable              :  0   N
  - doubly constrained        :  0   D
************************************************************************
PROJECT INFORMATION:
pronr.  #jobs rel.date duedate tardcost  MPM-Time
    1     30      0       31       24       31
************************************************************************
PRECEDENCE RELATIONS:
jobnr.    #modes  #successors   successors
   1        1          3           2   3   4
   2        1          3          11  12  15
   3        1          3           5   8  16
   4        1          1          27
   5        1          3           6  12  14
   6        1          1           7
   7        1          2           9  28
   8        1          2          10  20
   9        1          2          13  21
  10        1          2          17  25
  11        1          2          25  28
  12        1          2          25  29
  13        1          3          22  23  31
  14        1          1          17
  15        1          1          24
  16        1          1          18
  17        1          1          26
  18        1          2          19  23
  19        1          1          22
  20        1          1          29
  21        1          1          31
  22        1          1          24
  23        1          1          24
  24        1          1          29
  25        1          1          30
  26        1          1          28
  27        1          1          30
  28        1          1          30
  29        1          1          32
  30        1          1          32
  31        1          1          32
  32        1          0
************************************************************************
REQUESTS/DURATIONS:
jobnr. mode duration  R 1  R 2  R 3  R 4
------------------------------------------------------------------------
  1      1     0       0    0    0    0
  2      1     6       0    0    7    0
  3      1     4       0    0    1    0
  4      1     2       0    0    0    2
  5      1     1       0    0    6    0
  6      1     5       2    0    0    0
  7      1     1       0    0    0    6
  8      1     3       0    9    0    0
  9      1     5       0    8    0    0
 10      1     6       8    0    0    0
 11      1     6       0    0    7    0
 12      1     4       7    0    0    0
 13      1     1       8    0    0    0
 14      1     6       0    1    0    0
 15      1     5       0    0    0    2
 16      1     3       0    0    5    0
 17      1     3       0    2    0    0
 18      1     2       7    0    0    0
 19      1     3       0    8    0    0
 20      1     3       0    0    5    0
 21      1     7       0    0    0    2
 22      1     4       0    0    0    1
 23      1     8       0    0    0    5
 24      1     1       0    0    0    5
 25      1     8       0    9    0    0
 26      1     8       0    5    0    0
 27      1     5       0    9    0    0
 28      1     1       0    4    0    0
 29      1     5       0    4    0    0
 30      1     2       0    9    0    0
 31      1     1       0    0    0   10
 32      1     0       0    0    0    0
************************************************************************
RESOURCEAVAILABILITIES:
  R 1  R 2  R 3  R 4
   11   11    9   11
************************************************************************"""

import re

# Extract the number of tasks and resources
tasks_match = re.search(r'jobs \(incl\. supersource/sink \):\s*(\d+)', sm_content)
nb_tasks = int(tasks_match.group(1))

resources_match = re.search(r'- renewable\s*:\s*(\d+)', sm_content)
nb_resources = int(resources_match.group(1))

print(f"Number of tasks: {nb_tasks}")
print(f"Number of resources: {nb_resources}")

# Extract resource capacities
capacity_match = re.search(r'RESOURCEAVAILABILITIES:.*?R \d.*?\n(.*?)\n', sm_content, re.DOTALL)
capacities = list(map(int, capacity_match.group(1).strip().split()))
print(f"Resource capacities: {capacities}")

# Extract precedence relations
precedence_section = re.search(r'PRECEDENCE RELATIONS:.*?jobnr.*?\n(.*?)REQUESTS/DURATIONS', sm_content, re.DOTALL)
precedence_lines = precedence_section.group(1).strip().split('\n')

successors = {}
for line in precedence_lines:
    parts = line.strip().split()
    if not parts[0].isdigit():
        continue
    job_id = int(parts[0])
    num_successors = int(parts[2])
    succ_list = [int(parts[i]) for i in range(3, 3 + num_successors)]
    successors[job_id] = succ_list

# Extract durations and resource demands
durations_section = re.search(r'REQUESTS/DURATIONS:.*?jobnr.*?mode.*?\n-+\n(.*?)RESOURCEAVAILABILITIES', sm_content, re.DOTALL)
durations_lines = durations_section.group(1).strip().split('\n')

durations = {}
demands = {}
for line in durations_lines:
    parts = line.strip().split()
    if not parts[0].isdigit():
        continue
    job_id = int(parts[0])
    duration = int(parts[2])
    demand = [int(parts[i]) for i in range(3, 3 + nb_resources)]
    durations[job_id] = duration
    demands[job_id] = demand

print(f"First few tasks with durations: {list(durations.items())[:5]}")
print(f"First few tasks with demands: {list(demands.items())[:

In [ ]:
# Create the .data file content
data_content = []

# First line: number of tasks and resources
data_content.append(f"{nb_tasks} {nb_resources}")

# Second line: resource capacities
data_content.append(" ".join(map(str, capacities)))

# Rest of the file: one line per task
for job_id in range(1, nb_tasks + 1):
    duration = durations[job_id]
    demand = demands[job_id]
    succ = successors.get(job_id, [])

    # Format: duration demand1 demand2 ... demandN numSuccessors successor1 successor2 ...
    task_line = f"{duration} " + " ".join(map(str, demand)) + f" {len(succ)} " + " ".join(map(str, succ))
    data_content.append(task_line)

# Join all lines
data_file_content = "\n".join(data_content)

print("Generated .data file content:")
print(

In [ ]:
# Write to file
with open("j301_5.data", "w") as f:
    f.write(data_file_content)

print("File j301_5.data has been created successfully!")